In [12]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import shfl
import torch
import copy
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv
from ClassifierModel import ClassifierModel

In [13]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/COVIDGR1.0-cropped", 
        "csv_path": "../partitions/partition_iid_3nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 16,
        "federated_rounds": 1,
        "epochs_per_FL_round": 100,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", lambda_value = 0.00075, batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder():
    return ClassifierModel(batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

def get_transformed_data(federated_data, cit_federated_government, lb1, lb2):
    t_federated_data = copy.deepcopy(federated_data)

    for i in range(federated_data.num_nodes()):
        data_node = federated_data[i]
        t_data_node = t_federated_data[i]
        data = data_node.query()._data
        labels = data_node.query()._label
        t_data, t_labels = cit_federated_government.global_model.transform_data(data, labels, lb1, lb2)
        t_data_node.query()._data = t_data
        t_data_node.query()._label = t_labels

    t_test_data, t_test_label = cit_federated_government.global_model.transform_data(test_data, test_label, lb1, lb2)

    return t_federated_data, t_test_data, t_test_label

In [15]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

federated_data, train_data, train_label, test_data, test_label, train_files, test_files = get_federated_data_csv(args["data_path"], args["csv_path"], lb1)

cit_model = cit_builder()
cit_model.train(train_data, train_label)

t_train_data, t_train_label = cit_model.transform_data(train_data, train_label, lb1, lb2)
t_test_data, t_test_label = cit_model.transform_data(test_data, test_label, lb1, lb2)

classifier_model = classifier_builder()
classifier_model.train(t_train_data, t_train_label)

dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0])
              } 
G_dict = cit_model._G_dict

for key, _ in G_dict.items():
    G_dict[key].to("cpu")

classifier_model.get_classification_report(test_files, dict_labels, G_dict, "qaa")

/mnt/sdd/fcastro/TFGCOVID/src2/utils.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  federated_train_data = np.array(federated_train_data)
/mnt/sdd/fcastro/TFGCOVID/src2/utils.py:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  federated_train_label = np.array(federated_train_label)
  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] weights = [1.         0.95689655]


  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 31.75 GiB total capacity; 3.92 GiB already allocated; 3.75 MiB free; 165.57 MiB cached)